In [1]:
import pandas as pd

In [50]:
df = pd.read_csv('data/kc_house_data.csv')
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,NaN,NONE,...,7 Average,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650
1,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,NO,NONE,...,7 Average,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639
2,5631500400,2/25/2015,180000.0,2,1.00,770,10000,1.0,NO,NONE,...,6 Low Average,770,0.0,1933,NaN,98028,47.7379,-122.233,2720,8062
3,2487200875,12/9/2014,604000.0,4,3.00,1960,5000,1.0,NO,NONE,...,7 Average,1050,910.0,1965,0.0,98136,47.5208,-122.393,1360,5000
4,1954400510,2/18/2015,510000.0,3,2.00,1680,8080,1.0,NO,NONE,...,8 Good,1680,0.0,1987,0.0,98074,47.6168,-122.045,1800,7503


In [181]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 29 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    21597 non-null  int64  
 1   date                  21597 non-null  object 
 2   price                 21597 non-null  float64
 3   bedrooms              21597 non-null  int64  
 4   bathrooms             21597 non-null  float64
 5   sqft_living           21597 non-null  int64  
 6   sqft_lot              21597 non-null  int64  
 7   floors                21597 non-null  float64
 8   waterfront            19221 non-null  object 
 9   view                  21534 non-null  object 
 10  condition             21597 non-null  object 
 11  grade                 21597 non-null  object 
 12  sqft_above            21597 non-null  int64  
 13  sqft_basement         21597 non-null  object 
 14  yr_built              21597 non-null  int64  
 15  yr_renovated       

In [182]:
df.yr_renovated.value_counts()

0.0       17011
2014.0       73
2003.0       31
2013.0       31
2007.0       30
          ...  
1946.0        1
1959.0        1
1971.0        1
1951.0        1
1954.0        1
Name: yr_renovated, Length: 70, dtype: int64

In [194]:
df['yr_renovated'] = df['yr_renovated'].fillna(0)

In [51]:
df_condition = df.loc[:,['id','date','price','condition','grade','yr_built','yr_renovated']]

In [12]:
df_condition.describe()

,id,price,yr_built,yr_renovated
count,2.159700e+04,2.159700e+04,21597.000000,17755.000000
mean,4.580474e+09,5.402966e+05,1970.999676,83.636778
std,2.876736e+09,3.673681e+05,29.375234,399.946414
min,1.000102e+06,7.800000e+04,1900.000000,0.000000
25%,2.123049e+09,3.220000e+05,1951.000000,0.000000
50%,3.904930e+09,4.500000e+05,1975.000000,0.000000
75%,7.308900e+09,6.450000e+05,1997.000000,0.000000
max,9.900000e+09,7.700000e+06,2015.000000,2015.000000


In [13]:
df_condition.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            21597 non-null  int64  
 1   date          21597 non-null  object 
 2   price         21597 non-null  float64
 3   condition     21597 non-null  object 
 4   grade         21597 non-null  object 
 5   yr_built      21597 non-null  int64  
 6   yr_renovated  17755 non-null  float64
dtypes: float64(2), int64(2), object(3)
memory usage: 1.2+ MB


In [15]:
df_condition['condition'].value_counts()

Average      14020
Good          5677
Very Good     1701
Fair           170
Poor            29
Name: condition, dtype: int64

In [18]:
df_condition['grade'].value_counts().sort_index()

10 Very Good     1134
11 Excellent      399
12 Luxury          89
13 Mansion         13
3 Poor              1
4 Low              27
5 Fair            242
6 Low Average    2038
7 Average        8974
8 Good           6065
9 Better         2615
Name: grade, dtype: int64

In [57]:
def condition_coding (condition):
    """
    This will take the condition from string format and transform it into a corresponding code in integer format
    Poor = 1, Fair = 2, Average = 3, Good = 4, Very Good = 5
    """
    if condition == 'Poor':
        condition_code = 1
    elif condition == 'Fair':
        condition_code = 2
    elif condition == 'Average':
        condition_code = 3
    elif condition == 'Good':
        condition_code = 4
    elif condition == 'Very Good':
        condition_code = 5
    return condition_code

In [58]:
df['condition_code'] = df['condition'].map(condition_coding)

In [59]:
df.condition_code.head()

0    3
1    3
2    3
3    5
4    3
Name: condition_code, dtype: int64

In [70]:
def grade_coding (grade):
    """
    This takes the grade in string format, splits it into a list of characters
    It then concatenates the first two characters from the list and strips the whitespace and turns the result into an integer
    We are left with a one or two digit integer correspondinng to the grade of the property
    """
    grade_list = list(grade)
    grade_code = int((grade_list[0] + grade_list[1]).strip())
    return grade_code

In [71]:
df['grade_code'] = df['grade'].map(grade_coding)

In [72]:
df.grade_code.value_counts().sort_index()

3        1
4       27
5      242
6     2038
7     8974
8     6065
9     2615
10    1134
11     399
12      89
13      13
Name: grade_code, dtype: int64

In [75]:
df['age'] = 2022 - df['yr_built']

In [97]:
Renos = df.yr_renovated.value_counts()
sum(Renos)

17755

In [108]:
def renovated (year):
    if year == 0.0:
        return False
    elif year > 0.0:
        return True
    else:
        return False

In [109]:
df['Renovated'] = df['yr_renovated'].map(renovated)

In [111]:
df.Renovated.head(3)

0    False
1     True
2    False
Name: Renovated, dtype: bool

In [116]:
df['renovated_since_2000'] = df['yr_renovated'] >= 2000.0

In [117]:
df.renovated_since_2000.value_counts()

False    21218
True       379
Name: renovated_since_2000, dtype: int64

In [115]:
df.columns

Index(['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long', 'sqft_living15', 'sqft_lot15', 'condition_code',
       'grade_code', 'age', 'Renovated', 'renovated_since_2000'],
      dtype='object')

In [136]:
df['renovated_since_2010'] = df['yr_renovated'] >= 2010.0

In [135]:
len(df.id.unique())

21420

In [195]:
df['age_of_renovations'] = 2022 - df['yr_renovated']
df.loc[df['age_of_renovations'] == 2022.00, 'age_of_renovations'] = 0.0
df['age_of_renovations'] = df['age_of_renovations'].fillna(0)


In [196]:
df.age_of_renovations.value_counts()

0.0     20853
8.0        73
19.0       31
9.0        31
15.0       30
        ...  
71.0        1
72.0        1
78.0        1
51.0        1
74.0        1
Name: age_of_renovations, Length: 70, dtype: int64

In [193]:
df['price_per_sqft'] = df['price'] / df['sqft_living']

In [144]:
df.price_per_sqft.head()

0    188.050847
1    209.338521
2    233.766234
3    308.163265
4    303.571429
Name: price_per_sqft, dtype: float64

In [146]:
df.id.unique().sort_index()

AttributeError: 'numpy.ndarray' object has no attribute 'sort_index'

In [152]:
ids = []
repeats = []
for id in df['id']:
    if id not in ids:
        ids.append(id)
    else:
        repeats.append(id)
        
repeats
    

[6021501535,
 4139480200,
 7520000520,
 3969300030,
 2231500030,
 8820903380,
 726049190,
 8682262400,
 9834200885,
 8062900070,
 1231000510,
 6117501820,
 2228900270,
 3630120700,
 7888000390,
 2143700830,
 3395040550,
 4345000510,
 9407110710,
 1000102,
 1781500435,
 1232000810,
 7856400240,
 7983000200,
 9136103130,
 5083000375,
 7409700215,
 1823049202,
 2767602141,
 1254200015,
 723049156,
 4435000705,
 4364700600,
 1825069031,
 3262300940,
 9353300600,
 1524079093,
 9809000020,
 3303000130,
 7524400250,
 4204400098,
 6632900574,
 2726049071,
 3578401060,
 1788800630,
 6381500170,
 1954420170,
 1545800290,
 9828200460,
 9222400605,
 8832900780,
 8645530010,
 3323059027,
 5417600130,
 8651510380,
 5249801440,
 526059224,
 1446403850,
 251300110,
 4139420590,
 2621600015,
 2206700215,
 2787460720,
 3883800011,
 4154300296,
 1721801010,
 1237500540,
 5101405604,
 5127001320,
 5282200015,
 7977201065,
 3739500096,
 1568100300,
 1922059278,
 302000375,
 722039087,
 8129700644,
 4443800

In [153]:
len(repeats)

177

In [158]:
flips = df.loc[:,['id','price']]
dup_flips = flips[flips.duplicated(['id'])]

In [162]:
dup_flips.sort_index()

,id,price
94,6021501535,700000.0
314,4139480200,1400000.0
325,7520000520,240500.0
346,3969300030,239900.0
372,2231500030,530000.0
...,...,...
20165,7853400250,645000.0
20597,2724049222,220000.0
20654,8564860270,502000.0
20764,6300000226,380000.0


In [163]:
see = dup_flips.groupby(['id'])
see

In [166]:
print(see)

In [167]:
see.head()

,id,price
94,6021501535,700000.0
314,4139480200,1400000.0
325,7520000520,240500.0
346,3969300030,239900.0
372,2231500030,530000.0
...,...,...
20165,7853400250,645000.0
20597,2724049222,220000.0
20654,8564860270,502000.0
20764,6300000226,380000.0


In [170]:
dup_flips = dup_flips.merge(flips, how='left', on=['id'])

In [173]:
dup_flips.drop_duplicates(subset=['id'], inplace=True)

In [174]:
dup_flips

,id,price_x,price_y
0,6021501535,700000.0,430000.0
2,4139480200,1400000.0,1380000.0
4,7520000520,240500.0,232000.0
6,3969300030,239900.0,165000.0
8,2231500030,530000.0,315000.0
...,...,...,...
346,7853400250,645000.0,610000.0
348,2724049222,220000.0,163800.0
350,8564860270,502000.0,449990.0
352,6300000226,380000.0,240000.0


In [175]:
dup_flips['profit'] = dup_flips['price_x'] - dup_flips['price_y']

In [176]:
dup_flips

,id,price_x,price_y,profit
0,6021501535,700000.0,430000.0,270000.0
2,4139480200,1400000.0,1380000.0,20000.0
4,7520000520,240500.0,232000.0,8500.0
6,3969300030,239900.0,165000.0,74900.0
8,2231500030,530000.0,315000.0,215000.0
...,...,...,...,...
346,7853400250,645000.0,610000.0,35000.0
348,2724049222,220000.0,163800.0,56200.0
350,8564860270,502000.0,449990.0,52010.0
352,6300000226,380000.0,240000.0,140000.0


In [179]:
df.age_of_renovations.head(50)

0      NaN
1     31.0
2      NaN
3      NaN
4      NaN
5      NaN
6      NaN
7      NaN
8      NaN
9      NaN
10     NaN
11     NaN
12     NaN
13     NaN
14     NaN
15     NaN
16     NaN
17     NaN
18     NaN
19     NaN
20     NaN
21     NaN
22     NaN
23     NaN
24     NaN
25     NaN
26     NaN
27     NaN
28     NaN
29     NaN
30     NaN
31     NaN
32     NaN
33     NaN
34     NaN
35    20.0
36     NaN
37     NaN
38     NaN
39     NaN
40     NaN
41     NaN
42     NaN
43     NaN
44     NaN
45     NaN
46     NaN
47     NaN
48     NaN
49     NaN
Name: age_of_renovations, dtype: float64